In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import anodeclstmgru.constants as const
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact

In [ ]:
store = pd.HDFStore(const.HDF_STORE_PATH)
df_phys_norm_v1 = store['df_phys_norm_v1']
df_phys_att_v0 = store['df_phys_att_v0']
store.close()

In [ ]:
df_phys_norm_v1 = df_phys_norm_v1.set_index('Timestamp', drop=True)
df_phys_att_v0 = df_phys_att_v0.set_index('Timestamp', drop=True)
sensors = df_phys_att_v0.columns
day_list_normal = [f'2015-12-{d}' for d in range(22,28)] + ['all']
day_list_attack = [f'2015-12-{d}' for d in range(28,31)] + ['2016-01-01'] + \
                  ['2016-02-01'] + ['all']
sample_frequency_list = ['1s', '5s', '10s', '60s']
df_phys_norm_v1.head()

In [ ]:
def get_single_time_series(sensor, day='all', sample_freq='60s'):
    df_plot = df.copy()
    if not day == 'all':
        df_plot = df_plot[(df_plot.index > pd.Timestamp(day)) &
                         (df_plot.index < pd.Timestamp(day)+pd.Timedelta('1d'))]
    df_plot = df_plot.resample(sample_freq).mean()
    title = f'{sensor} over time (averaged over {sample_freq})'
    fig = px.line(df_plot, x=df_plot.index, y=sensor, title=title)
    fig.show()

In [ ]:
def get_4_signal_plot(sensor0='FIT101', sensor1='LIT101',
                      sensor2='LIT301', sensor3='LIT401', day='all', sample_freq='60s'):
    df_plot = df.copy()
    if not day == 'all':
        df_plot = df_plot[(df_plot.index > pd.Timestamp(day)) &
                         (df_plot.index < pd.Timestamp(day)+pd.Timedelta('1d'))]
    df_plot = df_plot.resample(sample_freq).mean()
    
    sensor_selection_list=[sensor0, sensor1, sensor2, sensor3]
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
    for i, s in enumerate(sensor_selection_list):
        fig.add_trace(
            go.Scatter(x=df_plot.index,
                       y=df_plot[s], name=s),
            row=i+1, col=1,
        )
    title = f'{sensor0}, {sensor1}, {sensor2} and {sensor3}'\
            f' over time (averaged over {sample_freq})'

    fig.update_layout(height=600, width=800, title_text=title)
    fig.show()

# Single time series during attack period

In [ ]:
df = df_phys_norm_v1.copy()
interact(get_single_time_series, sensor=sensors, day=day_list_normal,
         sample_freq=sample_frequency_list)

# Single time series during attack period

In [ ]:
df = df_phys_att_v0.copy()
interact(get_single_time_series, sensor=sensors, day=day_list_attack,
         sample_freq=sample_frequency_list)

# 4 time series during normal period

In [ ]:
df = df_phys_norm_v1.copy()
interact(get_4_signal_plot, sensor0=sensors, sensor1=sensors,
         sensor2=sensors, sensor3=sensors, day=day_list_normal,
         sample_freq=sample_frequency_list)

# 4 time series during attack period

In [ ]:
df = df_phys_att_v0.copy()
interact(get_4_signal_plot, sensor0=sensors, sensor1=sensors,
         sensor2=sensors, sensor3=sensors, day=day_list_attack,
         sample_freq=sample_frequency_list)

# Do we really have that gap in the timestamps?

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df_phys_att_v0.index.values)

In [ ]:
cols_norm = list(df_phys_norm_v1.columns)
cols_att = list(df_phys_att_v0.columns)

In [ ]:
set.difference(set(cols_nomr), set(cols_att))

In [ ]:
pd.DataFrame(dict(norm=sorted(df_phys_norm_v1.columns), att=sorted(df_phys_att_v0.columns)))